### Pytorch 모델 파일을 tensorflow 파일로 변환

아래 코드를 실행시켜 학습이 완료된 pytorch 모델파일(best.pt)을 tensorflow 모델파일(saved model)로 변환합니다.

>{가상환경 경로명}\Scripts\python.exe {yolov5-master 경로명}\export.py --weights {yolov5-master 경로명}\runs\train\exp\weights\best.pt --include saved_model

{yolov5-master 경로명}\runs\train\exp\weights 폴더로 들어가면 tensorflow 모델로 변환된 'saved_model' 폴더가 생성된 것을 확인할 수 있습니다.

'saved_model' 폴더를 실습파일이 있는 경로(현재 보고 있는 이 주피터 노트북 파일이 있는 경로)로 복사해 옵니다.

### 테스트 이미지를 입력으로 넣어 prediction하고 바운딩박스를 그려 이미지로 확인

In [1]:
import cv2
import numpy as np
import tensorflow as tf

1. tf로 변환된 모델(pb)을 로드해 모델 객체를 생성합니다.

In [2]:
#tf saved_model로 변환된 모델 파일 로드
model = tf.saved_model.load("backup/best_saved_model")

In [3]:
#모델의 입출력 구조 확인 → 입력은 (1, 640, 640, 3), 출력은 (1, 25200, 7)임을 알 수 있음
for k, v in model.signatures.items():
    print(k, v)

serving_default ConcreteFunction signature_wrapper(*, x)
  Args:
    x: float32 Tensor, shape=(1, 640, 640, 3)
  Returns:
    {'output_0': <1>}
      <1>: float32 Tensor, shape=(1, 25200, 7)


2. 이미지 하나를 로드해 모델에 입력으로 넣고 prediction 결과 행렬을 봅니다.

In [4]:
#테스트할 이미지를 행렬로 로드
src = cv2.imread("test_images/test_image.png")
src_converted = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
src_converted.shape # -> (2268, 4032, 3) 4032 x 2268 사이즈의 3채널 컬러이미지

(1080, 1920, 3)

In [5]:
#우리가 만든 모델은 (1, 640, 640, 3)을 입력으로 받으므로, 그에 맞게 resize & reshape을 진행
#입력에 차원이 하나 더 붙어있는 이유는 배치 때문으로, resize 후 reshape을 통해 변환
src_resized = cv2.resize(src_converted, (640, 640))
src_resized.shape # -> (640, 640, 3)

(640, 640, 3)

In [6]:
src_resized = src_resized.reshape((1, 640, 640, 3))
src_resized.shape # -> (1, 640, 640, 3)

(1, 640, 640, 3)

In [7]:
#추론 실행
src_resized = src_resized/255. #픽셀값이 0~1 사이 값으로 rescaling되어 트레이닝 되었음. 추론도 0~1 사이의 값으로 변환하고 시도해야 함
pred = model(src_resized)
pred # -> 리스트로 추론 결과가 묶여서 출력. 따라서 실제 추론결과는 pred[0]으로 접근해야 함

[<tf.Tensor: shape=(1, 25200, 7), dtype=float32, numpy=
 array([[[7.9063047e-03, 5.0242371e-03, 1.7675599e-02, ...,
          6.9760204e-06, 4.8929006e-01, 5.2223915e-01],
         [7.4648722e-03, 4.6702134e-03, 1.4114226e-02, ...,
          6.0116304e-06, 4.2623749e-01, 5.5148995e-01],
         [8.3673317e-03, 5.0265356e-03, 2.1602832e-02, ...,
          4.5579218e-06, 5.6351501e-01, 3.9660180e-01],
         ...,
         [9.8988801e-01, 9.8077792e-01, 1.3276212e-02, ...,
          6.9653647e-06, 2.8037885e-01, 6.4101899e-01],
         [9.9130201e-01, 9.8380512e-01, 4.2167697e-02, ...,
          2.3403245e-06, 2.4748440e-01, 6.6107756e-01],
         [9.8839682e-01, 9.8233902e-01, 6.5405197e-02, ...,
          1.1161677e-06, 1.7683490e-01, 6.7621481e-01]]], dtype=float32)>]

In [8]:
pred[0].shape # -> TensorShape([1, 25200, 7])  *numpy가 아니라 tf 프레임웍 포맷에 맞춰 출력

TensorShape([1, 25200, 7])

In [9]:
"""
결과물 TensorShape([1, 25200, 7])의 제일 첫 차원은 의미 없고 나머지 차원이 중요 -> 25200행 * 7열로 이해할 것
25200행 : 입력에 따라 추론한 25200개의 객체검출 결과
7열 : 행별 결과내용 → [cx, cy, w, h, obj conf, c1 conf, c2 conf]
  - 0~3번째 열은 검출한 바운딩 박스 정보
  - 4번째 열은 바운딩 박스 안에 객체가 있을 확률
  - 5번째 열은 검출한 객체가 1번 클래스(도로표지판)일 확률
  - 6번째 열은 검출한 객체가 2번 클래스(신호등)일 확률
"""

pred = pred[0][0].numpy() #작업 용이성을 위해 필요없는 첫 차원은 제거하고 (25200, 7)만 선택, numpy 행렬로 변환한 후 pred에 덮어 씌움
pred.shape # -> (25200, 7)

(25200, 7)

3. 추론한 바운딩 박스 정보를 원본 이미지에 그린 후 결과를 봅니다.

In [10]:
#25200개를 모두 화면에 그리면 너무 복잡함. obj conf. 순으로 정렬한 후 상위 3000개만 뽑아서 출력해 본다.
#pred의 4번째 열(-> obj conf.)을 기준으로 내림차순 정렬
rank = np.argsort(-pred[:, 4]) # -> array([24119, 24116, 24113, ...,  2613,  1893,  1653], dtype=int64)
pred_sorted = pred[rank]

dst = cv2.resize(src, (int((900*src.shape[1])/src.shape[0]), 900), interpolation=cv2.INTER_LINEAR) #이미지가 너무 크므로 리사이즈한다.
for bbox in pred_sorted[:3000]:
    cx = int(bbox[0] * dst.shape[1])
    cy = int(bbox[1] * dst.shape[0])
    w = int(bbox[2] * dst.shape[1])
    h = int(bbox[3] * dst.shape[0])
    x = int(cx - (w/2))
    y = int(cy - (h/2))

    dst = cv2.rectangle(dst, (x, y, w, h), (0, 255, 0), 2)
cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

결과를 보면 오브젝트가 있는 곳 주변으로 바운딩박스가 많이 겹쳐져 나온다는 것을 알 수 있습니다. 겹쳐져 있는 바운딩박스를 처리하기 위한 Non Maximun Suppresion 작업이 필요하다는 것을 알 수 있습니다.